In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from obspy import *
from obspy.core import read, UTCDateTime
from obspy.clients.fdsn import Client
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
import numpy as np
import os
from os import path
import matplotlib.pylab as plt
from plotw_rs import plotw_rs
import cartopy.feature as cfeature
from plot_event_station import plot_event_station
from obspy.geodetics import kilometers2degrees, gps2dist_azimuth, degrees2kilometers
from geopy import distance
from scipy.interpolate import interp2d

# sumatra_hf_template.m
# Applied Seismology, GEOS 626, University of Alaska Fairbanks
# 
# Template script for analyzing the direct arrival waveforms from Sumatra:
#     channel BHZ, duration of up to 2 hours
#
# Calls plotw_rs

# Plotting parameter
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

In [ ]:
spdy = 86400;   # seconds per day

# extract the full database of BHZ waveforms
otimePDE   = UTCDateTime("2004-12-26T00:58:53.0")
originTime = otimePDE;
startTime  = originTime - 5*60
endTime    = originTime + 2*60*60
elat = 3.09
elon = 94.26
edep_km = 28.6
BHdirectory = "./datawf/sumatraBH"
client = Client("IRIS")
datadir = "./data"
iprocess = True       #deconvolve instrument response if True

In [ ]:
# max radius for station search
stasub = [0,180] #in degrees

stanames=[]
# Load pre-saved BHZ stations 
w=Stream()
wc=read(BHdirectory+"/*Z", header=None)
#print(wc)
wc.merge(method=1, fill_value=0)
inventory = client.get_stations(minradius=stasub[0], maxradius=stasub[1], longitude=elon, 
                    latitude=elat, network="G,II,IU", starttime=startTime, endtime =endTime,  
                    channel="BHZ", level="response")

print(inventory)
for net in inventory:     
    for sta in net:
        for cha in sta:
            if str(sta.code)+str(cha.location_code) not in stanames:
                # Use inventory to save station locations
                try:
                    ws=wc.select(station=sta.code, location=cha.location_code)
                    ws[0].trim(originTime-(5*60),originTime+(2*3600), pad=True, fill_value=0)
                    ws[0].stats.sampling_rate=round(ws[0].stats.sampling_rate)
                    ws[0].stats.sac.stla=sta.latitude
                    ws[0].stats.sac.stlo=sta.longitude
                    if iprocess == True:
                        ws[0].remove_response(inventory=inventory, water_level=20) #user: consider changing variables 
                    w=w.append(ws[0])
                    stanames.append(str(sta.code)+str(cha.location_code))
                except:
                    pass
wc=[]                
# Add station location data to the Obspy Trace
nw=len(w)
print('here is a list of the waveforms you have:');
for ii in range(nw):
    print('%3i %7s %3s %6s' % (ii,w[ii].stats.channel,w[ii].stats.network,w[ii].stats.station))

# save a copy to avoid rerunning
wpick=Stream()
# pick a subset of waveforms
#ipick = np.arange(nw)                   # default use all waveforms
ipick = [36,1,61,49,50,159]     # USER: CHANGE THIS 
for i in ipick:
    wpick.append(w[i])

In [ ]:
# PLOTTING PARAMETERS FOR plotw_rs.m (USER: CHANGE THESE AS NEEDED)
rssort = 2      # =1 by azimuth, =2 by distance
iabs = 0
tshift = []
tmark = []
pmax = 40
iintp = 0
inorm = [1]
tlims = []
nfac = 1
azstart = []
iunit = 2
imap = 0

# plot record section
T1 = [];
T2 = [];
# USER: add necessary arguments if they are different from the default
plotw_rs(wpick,elat,elon,rssort,iabs,tshift,tmark,T1,T2,pmax,iintp,inorm,tlims,
                 nfac,azstart,iunit,imap)

In [ ]:
# Plot global map of selected stations
plot_event_station(elat,elon,w=wpick)

In [ ]:
# SOME EXAMPLES OF USING THE PLOTTING COMMANDS
# example of cutting a record

iremove=[3,4]
wp=wpick.copy()
for r in iremove:
    print(wp[r])
    wp.remove(wp[r]) # cut a bad record (WAKE) and a repeated record (KDAK)
plotw_rs(wp,elat,elon,rssort,iabs,tshift,tmark,T1,T2,pmax,iintp,inorm,tlims,
                 nfac,azstart,iunit,imap)


In [ ]:
# arc distances to stations
dist_deg=[]

for i in range(len(wp)):
    dis=distance.distance((elat, elon), (wp[i].stats.sac.stla, wp[i].stats.sac.stlo)).km
    arcdeg=kilometers2degrees(dis)                      
    dist_deg.append(arcdeg)
# OPTION A: use the Jeffreys-Bullen traveltime tables
# WARNING: our simplified version only works for epicentral distances <100 deg

h=edep_km
delta= dist_deg
print(delta)
def get_JB_Ptime(h,delta):

    '''
    INPUT:    h       source depth, km
               delta   arc distance, degrees
     OUTPUT:   t       direct P-wave travel time from Jeffreys-Bullen table, seconds
     WARNING: This simplistic function only considers for direct P, which
              is not present for arc distances above 100 deg.
    
     load Jeffreys-Bullen table for P'''
    
    jbP = np.loadtxt(datadir+'/jbP.txt', skiprows=3, dtype=float)        # Skip lines, 0,1,2
    # full table
    ndep= len(jbP[0,:])-1
    h0=[]
    delta0=[]
    # interpolate the table
    for i in range(ndep+1):
        if i>0:
            h0.append(jbP[0, i])
    for i in range(len(jbP[:,0])):
        if i>0:
            delta0.append(jbP[i,0])
    jbP = np.delete(jbP, (0), axis=0)
    jbP = np.delete(jbP, (0), axis=1)
    xx, yy = np.meshgrid(h0, delta0)
    z = np.sin(xx**2+yy**2)
    f = interp2d(delta0, h0, jbP.T)
    Ptt=[]
    for i in range(len(delta)):
        Ptt.append(f(delta[i],h))
    return Ptt              
    
Ptt = get_JB_Ptime(edep_km,dist_deg)
for i in range(len(Ptt)):
    if np.isnan(Ptt[i]):
        print('WARNING: JB times are NaN, since no direct P for Delta > 100 deg')
        Ptt[i] = [1000]    # dummy arrival time (use tauP in the future!)'''
## OPTION B: example of applying a relative time shift to each seismogram
## Note: This is in the order of listed stations (NOT as ordered in the record section).
## Note: The DT are w.r.t. the origin time and are listed on the labels.
# get(w,'station')
# tshift = [1186, 1250, 845, 1440];
# plotw_rs(wpick,elat=elat,elon=elon,rssort=rssort,iabs=iabs,tshift=tshift,tmark=tmark,T1=T1,T2=T2,
#         pmax=pmax,iintp=iintp,inorm=inorm,nfac=nfac,iunit=iunit,imap=imap)
# 
## OPTION C: assume a constant velocity (as suggested in the homework)
# Vest = 11;                      % km/s
# Ptt = degrees2kilometers(dist_deg) / Vest;  % very crude estimation for 30 < Delta < 85

# now replot
# Here the time shift is relative to the marker time (originTime),
# so the DT in the record section is the predicted P travel time (Ptt).
tshiftmark = Ptt
tmark = [originTime]
plotw_rs(wp,elat,elon,rssort,iabs,tshiftmark,tmark,T1,T2,pmax,iintp,inorm,tlims,
                 nfac,azstart,iunit,imap)

In [ ]:
# example of resetting plotting range
# Note that the amplitude scaling is based on the full-length seismogram,
# not the (subset) time interval that is plotted.
tlims = [-50, 700]
plotw_rs(wp,elat,elon,rssort,iabs,tshiftmark,tmark,T1,T2,pmax,iintp,inorm,tlims,
                 nfac,azstart,iunit,imap)

In [ ]:
# START YOUR ANALYSIS HERE